## Demo: Run Multiple Intent Experiments

In [1]:
!rm -rf ./test

In [2]:
%%capture
from leaderboard.nlu.base import base_experiment, intent_experiment
import copy

### Chose Configurations

In [3]:
base_config = intent_experiment.IntentExperimentConfiguration(
    exclusion_percentage=0.,
    drop_intents_with_less_than=2,
    data=base_experiment.DataConfig(
        name="examples-rules-nlu",
        data_path="../../examples/rules/data/nlu.yml",
        domain_path="../../examples/rules/domain.yml",
    ),
    model=base_experiment.ModelConfig(),
    clear_rasa_cache=True,
)

In [4]:
variations = {
    "interesting-config-name1" : "./config2.yml" ,
    "another-very-interesting-config-because-reasons" : "./config1.yml" ,
}

In [5]:
configs = []
for name, config_path in variations.items():
    config = copy.deepcopy(base_config)
    config.model.name = name
    config.model.config_path = config_path
    configs.append(config)

In [6]:
configs

[IntentExperimentConfiguration(description='Basic Intent Experiment', model=ModelConfig(name='interesting-config-name1', config_path='./config2.yml'), data=DataConfig(name='examples-rules-nlu', domain_path='../../examples/rules/domain.yml', data_path='../../examples/rules/data/nlu.yml'), clear_rasa_cache=True, drop_intents_with_less_than=2, exclusion_percentage=0.0, random_seed=1234),
 IntentExperimentConfiguration(description='Basic Intent Experiment', model=ModelConfig(name='another-very-interesting-config-because-reasons', config_path='./config1.yml'), data=DataConfig(name='examples-rules-nlu', domain_path='../../examples/rules/domain.yml', data_path='../../examples/rules/data/nlu.yml'), clear_rasa_cache=True, drop_intents_with_less_than=2, exclusion_percentage=0.0, random_seed=1234)]

### Run!

In [7]:
!ls -lh ./

total 112
-rw-r--r--  1 kabu  staff    12K Apr  4 13:08 Example1_Run-Multiple-Intent-Experiments.ipynb
-rw-r--r--  1 kabu  staff    25K Apr  4 13:49 Example2_Run-Exp0-Script.ipynb
-rw-r--r--  1 kabu  staff     0B Mar 31 19:21 __init__.py
-rw-r--r--  1 kabu  staff   213B Mar 31 17:41 config1.yml
-rw-r--r--  1 kabu  staff   176B Apr  1 14:33 config2.yml
-rw-r--r--  1 kabu  staff   1.1K Apr  4 12:54 run_exp0.py
drwxr-xr-x  3 kabu  staff    96B Apr  4 13:04 test2


In [8]:
intent_experiment.multirun(configs=configs, out_dir='./test', capture=True) # capture=True captures all stdout/stderr output

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:51<00:00, 55.53s/it]


### Inspect the Results

In [9]:
!ls -lh ./test/intent_experiment__examples-rules-nlu

total 0
drwxr-xr-x  7 kabu  staff   224B Apr  4 13:51 config:another-very-interesting-config-because-reasons,drop:2,exclude:0.0,seed:1234__2022-04-04_13-50-35
drwxr-xr-x  7 kabu  staff   224B Apr  4 13:50 config:interesting-config-name1,drop:2,exclude:0.0,seed:1234__2022-04-04_13-49-40


In [10]:
import glob
example = list(glob.glob("./test/intent_experiment__examples-rules-nlu/*"))[0]

In [11]:
example

'./test/intent_experiment__examples-rules-nlu/config:interesting-config-name1,drop:2,exclude:0.0,seed:1234__2022-04-04_13-49-40'

In [12]:
!ls -lha {example}

total 8
drwxr-xr-x  7 kabu  staff   224B Apr  4 13:50 .
drwxr-xr-x  4 kabu  staff   128B Apr  4 13:50 ..
drwxr-xr-x  5 kabu  staff   160B Apr  4 13:49 .hydra
drwxr-xr-x  4 kabu  staff   128B Apr  4 13:50 data
-rw-r--r--  1 kabu  staff   2.5K Apr  4 13:50 intent_experiment.log
drwxr-xr-x  3 kabu  staff    96B Apr  4 13:50 model
drwxr-xr-x  7 kabu  staff   224B Apr  4 13:50 report


In [13]:
!head -n 5 {example}/intent_experiment.log

[2022-04-04 13:49:40,809][leaderboard.nlu.base.base_experiment][INFO] - Running Experiment IntentExperiment
[2022-04-04 13:49:40,809][leaderboard.nlu.base.base_experiment][INFO] - Description: Experiment where we 1. exclude all intents with less than a fixed number of examples and 2. during training also remove a fixed percentage of the training examples before training.
[2022-04-04 13:49:41,659][rasa.engine.training.hooks][INFO] - Starting to train component 'CountVectorsFeaturizer'.
[2022-04-04 13:49:41,712][rasa.nlu.featurizers.sparse_featurizer.count_vectors_featurizer][INFO] - 232 vocabulary items were created for text attribute.
[2022-04-04 13:49:41,715][rasa.engine.training.hooks][INFO] - Finished training component 'CountVectorsFeaturizer'.


In [14]:
!tail -n 5 {example}/intent_experiment.log

 [ 0  0  2  1  2 17  1  1  2]
 [ 0  0  0  1  1  1  0  0  1]
 [ 0  0  2  0  1  0  0  0  1]
 [ 0  0  1  1  0  1  0  0  2]]
[2022-04-04 13:50:30,059][rasa.nlu.test][INFO] - Entity evaluation results:


In [15]:
!cat {example}/.hydra/config.yaml

description: Basic Intent Experiment
model:
  name: interesting-config-name1
  config_path: ./config2.yml
data:
  name: examples-rules-nlu
  domain_path: ../../examples/rules/domain.yml
  data_path: ../../examples/rules/data/nlu.yml
clear_rasa_cache: true
drop_intents_with_less_than: 2
exclusion_percentage: 0.0
random_seed: 1234


In [16]:
!head -n 5 {example}/.hydra/hydra.yaml
!echo '...'

hydra:
  run:
    dir: ./test/intent_experiment__${data.name}/config:${model.name},drop:${drop_intents_with_less_than},exclude:${exclusion_percentage},seed:${random_seed}__${now:%Y-%m-%d_%H-%M-%S}
  sweep:
    dir: multirun/${now:%Y-%m-%d}/${now:%H-%M-%S}
...


In [17]:
!ls {example}/report/

intent_confusion_matrix.png  intent_report.json
intent_errors.json           training_metadata__times.csv
intent_histogram.png


In [18]:
!cat {example}/report/training_metadata__times.csv

name, start_time, duration(min), duration(sec)
train_CountVectorsFeaturizer1, 2022-04-04 13:49:41, 0.0002, 0.01269388198852539
train_DIETClassifier2, 2022-04-04 13:49:41, 0.3565, 21.389381170272827
